In [26]:
import os
import pandas as pd
#os.chdir("/Users/gunjan/Desktop/Rahul/backtest/Data")
os.chdir("/Users/gunjan/Desktop/Youtube/video2/Data")

In [30]:
files = os.listdir()
len(files)

52

In [31]:
list1 = []

In [32]:
for i in range(0,len(files)):
    import pandas as pd
    #i = 0
    if files[i] != '.DS_Store':
        file = pd.read_csv(files[i])
        file["Date"] = pd.to_datetime(file["Date"])
        
        #print(files[i])
        file = file[["Date","Open","Close","Low","High"]]
        #print(files[i],file.head(2))
        df = file.sort_index()
        df['9ema'] = df['Close'].ewm(span=9,min_periods=0,adjust=False,ignore_na=False).mean()
        df['16ema'] = df['Close'].ewm(span=16,min_periods=0,adjust=False,ignore_na=False).mean()
        
        delta = df['Close'].diff()
        up = delta.clip(lower=0)
        down = -1*delta.clip(upper=0)
        ema_up = up.ewm(com=13, adjust=False).mean()
        ema_down = down.ewm(com=13, adjust=False).mean()
        rs = ema_up/ema_down

        df['RSI'] = 100 - (100/(1 + rs))
        #df["RSI"] = df.apply(RSI,axis=1)
        #df = df[(df['Date'] >= '2008-01-01')]
        def emacross(s):
            if (s['9ema'] > s['16ema']):
                return 1
            else:
                return 0
        if(df.shape[0] == 0):
            pass
        else:
            df['trade'] = df.apply(emacross, axis=1)
            #df['cum_trade'] = df.groupby('trade')["trade"].cumsum()
            #df['cum_trade'] = df.trade.astype('category').cat.codes.add(1)
            #df = df["trade"].lag()

            df['cum_trade'] = (df.trade.diff()!=0).cumsum()

            df_buy = df.groupby('cum_trade')
            df_buy = df_buy.nth(1)
            df_buy = df_buy[(df_buy["trade"] == 1)]
            #df_buy = df_buy.drop(["Date","High","Low","Close","Adj Close","Volume","9ema","16ema","trade"],axis=1)
            #df_buy = df_buy.drop(["Date","High","Low","Close","Adj Close","Volume","9ema","16ema","trade"],axis=1)
            df_buy = df_buy[['Open']]
            df_buy["cum_trade"] = df_buy.index
            df_buy.columns = ['buy', 'cum_trade']
            df_buy = df_buy.reset_index(level=0, drop=True)

            df_sell = df.groupby('cum_trade')
            df_sell = df_sell.nth(1)
            df_sell = df_sell[(df_sell["trade"] == 0)]
            #df_sell = df_sell.drop(["Date","High","Low","Close","Adj Close","Volume","9ema","16ema","trade"],axis=1)
            df_sell = df_sell[['Open']]
            df_sell["cum_trade"] = df_sell.index
            df_sell = df_sell[(df_sell["cum_trade"] > 1)]
            df_sell["cum_trade"] = df_sell["cum_trade"]-1
            #df = df.drop(["cum_trade1"],axis=1)
            df_sell.columns = ['sell', 'cum_trade']
            #df_sell = df_sell.reset_index
            df_sell = df_sell.reset_index(level=0, drop=True)

            df  = df.merge(df_buy).merge(df_sell)

            import numpy as np
            def mask_first(x):
                result = np.ones_like(x)
                result[0] = 0
                return result

            mask = df.groupby(['cum_trade'])['cum_trade'].transform(mask_first).astype(bool)
            df = df.loc[mask]
            df["loss_percentage"] = (df["Low"] - df["buy"])*100/df["buy"]
            df["profit_per"] = (df["High"] - df["buy"])*100/df["buy"]

            d = [('loss','min')]
            df = df.join(df.groupby(['cum_trade'])['loss_percentage'].agg(d), on=['cum_trade'])
            d = [('gain','max')]
            df = df.join(df.groupby(['cum_trade'])['profit_per'].agg(d), on=['cum_trade'])

            df = df.groupby('cum_trade').first()
            df["profit"] = (df["sell"] - df["buy"])*100/df["buy"]
            #df = df[(df["RSI"] > 50)]
            def sell(x):
                if x["loss"] < -5:
                    return -5
                elif x["profit"] < -5:
                    return -5
                else:
                    return x["profit"]
            def count_1(x):
                if x["status"] > 1:
                    return 1
                else:
                    return 0
            if df.shape[0] == 0:
                pass
            else:
                df["status"] = df.apply(sell, axis=1)
                df["count_"] = df.apply(count_1, axis=1)
                print(files[i])
                print(df.shape[0])
                list1.append([files[i],sum(df["status"]),sum(df["count_"])/df.shape[0],sum(df["status"])/df.shape[0],df.shape[0]])
                
                
                

BHARTIARTL.csv
96
TATASTEEL.csv
106
TITAN.csv
102
BRITANNIA.csv
104
MM.csv
118
COALINDIA.csv
59
JSWSTEEL.csv
76
NESTLEIND.csv
57
ICICIBANK.csv
118
MARUTI.csv
87
ULTRACEMCO.csv
85
WIPRO.csv
114
NIFTY50_all.csv
4930
NTPC.csv
100
VEDL.csv
102
ASIANPAINT.csv
109
ONGC.csv
120
IOC.csv
126
DRREDDY.csv
113
TECHM.csv
70
TCS.csv
99
SUNPHARMA.csv
117
HCLTECH.csv
118
HDFC.csv
122
KOTAKBANK.csv
105
ZEEL.csv
115
AXISBANK.csv
114
EICHERMOT.csv
114
HDFCBANK.csv
114
TATAMOTORS.csv
114
HEROMOTOCO.csv
119
UPL.csv
88
CIPLA.csv
119
BAJAJ-AUTO.csv
60
ITC.csv
123
GAIL.csv
119
POWERGRID.csv
80
HINDALCO.csv
108
BPCL.csv
114
LT.csv
69
ADANIPORTS.csv
69
HINDUNILVR.csv
109
GRASIM.csv
105
RELIANCE.csv
115
INFY.csv
113
SHREECEM.csv
90
SBIN.csv
111
INDUSINDBK.csv
99
BAJAJFINSV.csv
65
BAJFINANCE.csv
83


In [33]:
df = pd.DataFrame(list1, columns = ['filename', 'netPL', 'winratio', 'PLpertrade', "#trade"])


In [34]:
#df.to_csv("backtest_results.csv")

In [35]:
df

,filename,netPL,winratio,PLpertrade,#trade
0,BHARTIARTL.csv,252.463518,0.322917,2.629828,96
1,TATASTEEL.csv,656.425410,0.330189,6.192693,106
2,TITAN.csv,261.350670,0.333333,2.562261,102
3,BRITANNIA.csv,257.114176,0.298077,2.472252,104
4,MM.csv,300.639930,0.271186,2.547796,118
5,COALINDIA.csv,-83.178584,0.220339,-1.409807,59
6,JSWSTEEL.csv,419.708358,0.342105,5.522478,76
7,NESTLEIND.csv,109.250906,0.421053,1.916683,57
8,ICICIBANK.csv,217.073420,0.245763,1.839605,118
9,MARUTI.csv,523.451310,0.344828,6.016682,87


In [36]:
df["netPL"].mean()

530.2330067177688

In [37]:
df["winratio"].mean()

0.3009792190369963

In [38]:
df['PLpertrade'].mean()

2.4555768897592607

In [39]:
df["#trade"].sum()

9882